In [ ]:
!git clone https://github.com/fadamsyah/computer-vision-with-pytorch.git

In [ ]:
cd computer-vision-with-pytorch/Segmentation

In [ ]:
import zipfile

with zipfile.ZipFile('../../../input/carvana-image-masking-challenge/train.zip',"r") as zip_ref:
    zip_ref.extractall(".")
    
with zipfile.ZipFile('../../../input/carvana-image-masking-challenge/train_masks.zip',"r") as zip_ref:
    zip_ref.extractall(".")
    
with zipfile.ZipFile('../../../input/carvana-image-masking-challenge/train_masks.csv.zip',"r") as zip_ref:
    zip_ref.extractall(".")
    
with zipfile.ZipFile('../../../input/carvana-image-masking-challenge/metadata.csv.zip',"r") as zip_ref:
    zip_ref.extractall(".")

In [ ]:
# TPU: Still fail
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev
# !pip install pytorch-lightning==1.1.8

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import albumentations as A

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

import os
import sys

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from albumentations.pytorch import ToTensorV2

from lib.pl_datamodules import CarvanaDataModule
from lib.pl_models import SegmentationModel

%matplotlib inline

In [ ]:
# Initialize the transformation of training and validation dataset
transform = {
    'train': A.Compose([
        A.Resize(256, 256, always_apply=True),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(scale_limit=0.2, rotate_limit=10., shift_limit=0.1, p=1., border_mode=0),
        A.OneOf([
            A.IAASharpen(p=1),
            A.Blur(blur_limit=3, p=1),
            A.MotionBlur(blur_limit=3, p=1)
        ], p=0.9),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]),
    'val': A.Compose([
        A.Resize(256, 256, always_apply=True),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
}

In [ ]:
# Initialize a data module
dm = CarvanaDataModule('train', 'train_masks', 0.2, transform)
dm.configure_train_dataloader(batch_size=32)
dm.configure_val_dataloader(batch_size=32)

# Initialize a model
model = SegmentationModel()
model.freeze_encoder()

# Initialize a trainer
trainer = pl.Trainer(gpus=1, max_epochs=10, progress_bar_refresh_rate=1,
                     precision=16, amp_level='O2', benchmark=True)
# trainer = pl.Trainer(tpu_cores=8, max_epochs=5, progress_bar_refresh_rate=20, precision=16)

In [ ]:
# Train the model
trainer.fit(model, dm)